# Monk1 
Grid Search for Monk1 dataset model selection


In [1]:
from exclusiveAI.ConfiguratorGen import ConfiguratorGen
from exclusiveAI.datasets.monk import read_monk1
from exclusiveAI.components.Validation import *
from exclusiveAI.utils import one_hot_encoding
from exclusiveAI.components.CallBacks import *
from exclusiveAI.components.Initializers import *
from exclusiveAI.Composer import Composer
import pandas as pd
import numpy as np
import pickle
import os

#### Read Monk1 dataset 

In [2]:
training_data, training_labels, test_data, test_labels = read_monk1("../exclusiveAI/datasets/")

One-Hot Encoding Training Dataset

In [3]:
training_data = one_hot_encoding(training_data)

One-Hot Encoding Test Dataset

In [4]:
test_data = one_hot_encoding(test_data)

Early Stopping Callback with 50 epochs patience limit

In [5]:
patience_limit = 10
early_stopping = EarlyStoppingCallback(patience_limit=patience_limit, eps=1e-3)

Configuration Generator Grid Search for better hyperparameters selection

In [6]:
regularizations = np.arange(0, 0.001, 0.0001)
learning_rates = np.arange(0.01, 0.5, 0.01)
number_of_units = range(1, 5, 1)
number_of_layers = range(1, 3, 1) 
uniform = Uniform(low=-1, high=1)
initializers = ["uniform", "gaussian"]
momentums = np.arange(0, 0.999, 0.001)
activations = ["sigmoid"]
file_name = "monk1_configurations.pkl"

if not os.path.exists(file_name):
    myConfigurator = ConfiguratorGen(random=False, regularizations=regularizations, learning_rates=learning_rates,
                                     loss_function=['mse'], optimizer=['sgd'],
                                     activation_functions=activations, nesterov=True,
                                     number_of_units=number_of_units, number_of_layers=number_of_layers,
                                     momentums=momentums, initializers=initializers, input_shapes=training_data.shape, verbose=False,
                                     callbacks=[early_stopping], output_activation='sigmoid'
                                     )
    
    myConfigurator.save(file_name)
else: myConfigurator = pickle.load(open(file_name, 'rb'))

HoldOut validation with our configurations

In [7]:
batch_size=32
epochs=200

myValidation = HoldOut(models=myConfigurator, input=training_data, target=training_labels, debug=True)
configs = myValidation.hold_out(all_models=True, batch_size=batch_size, epochs=epochs, disable_line=False)

Epochs:   6%|▌         | 11/200 [00:00<00:00, 573.28it/s, mse=0.256, val_mse=0.248, mae=0.501, val_mae=0.492, mee=0.501, val_mee=0.492, binary_accuracy=0.495, val_binary_accuracy=0.52]

Models:   0%|          | 1/39160800 [00:19<214801:57:17, 19.75s/it]


KeyboardInterrupt: 

In [ ]:
configs = pd.DataFrame(configs)
# Save as json
configs.to_json('monk1_models_configurations.json')